In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.preprocessing import OneHotEncoder
from keras.applications.resnet import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Conv2D, Flatten
from keras.callbacks import TensorBoard
from tensordash.tensordash import Tensordash
import tensorflow as tf
import time

# Data preprocessing

In [ ]:
table=pd.read_csv("fer2013.csv").values
x=list()
for i in range(table.shape[0]):
    table[i][1]=np.uint8(np.asarray(table[i][1].split(' ')).reshape((48,48,1)))
    im=table[i][1]
    im=cv2.merge([im,im,im])
    x.append(im)
    print("Current row : %d"%i,end="\r")
print("\nDone!")
table=table[:,:-1]
x=np.float32(np.asarray(x))/255.0
y=np.uint8(table[:,0])
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
y = onehot_encoder.fit_transform(y.reshape(-1,1))
np.save("xval.npy",x)
np.save("yval.npy",y)
np.save("data.npy",table)

In [ ]:
x=np.load('xval.npy')
y=np.load('yval.npy')

In [ ]:
table=np.load('data.npy', allow_pickle=True)

In [ ]:
image=x[random.randint(0,x.shape[1]-1)]
plt.imshow(np.asarray(image.reshape(48,48,3)))
plt.show()

In [ ]:
resnet=ResNet50(weights='imagenet', include_top=False, input_shape=(x.shape[1],x.shape[2],3))
resnet.summary()

In [ ]:
# model.add(Input(shape=(48,48,1)))
# model.add(Conv2D(filters=3,kernel_size=3,padding='same'))
mx=resnet.output
mx=Flatten()(mx)
mx=Dense(1000,activation='relu')(mx)
mx=Dense(y.shape[1],activation='softmax')(mx)
model=Model(resnet.input,mx)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
tensorboard=TensorBoard(log_dir="log/%f"%(time.time()))
tensorboard.set_model(model)

In [ ]:
histories=Tensordash(email='...', password='...', ModelName='Facial Emotion Recognition')

In [ ]:
with tf.device('/gpu:0'):
    history=model.fit(x,y,epochs=10,batch_size=128,validation_split=0.1,shuffle=True,verbose=1,callbacks=[tensorboard, histories])

In [ ]:
model.save("detect.h5")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Validation Accuracy'], loc='upper left')
plt.savefig("Accuracy.png")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Loss', 'Validation Loss'], loc='upper right')
plt.savefig("Loss.png")
plt.show()

KeyboardInterrupt: 